**Семинар 2.**

Пусть у нас имеется некоторый набор данных sales.csv о продажах квартир.

— цена продажи (SalePrice, зависимая переменная),

— наземная жилая зона (GrLivArea),

— общее качество (Overall Quality),

— территория гаража (GarageArea),

— условия сделки (SaleCondition).

Реализуйте построение модели линейной регрессии на этих данных. Проверьте качество модели на обучающей и тестовой выборках с помощью MAE, MSE.

In [66]:
import pandas as pd
df = pd.read_csv('sales.csv')
# Display the first few rows of the DataFrame
print(df.head())

   SalePrice  GrLivArea  GarageArea  OverallQual Street SaleCondition
0     208500       1710         548            7   Pave        Normal
1     181500       1262         460            6   Pave        Normal
2     223500       1786         608            7   Pave        Normal
3     140000       1717         642            7   Pave       Abnorml
4     250000       2198         836            8   Pave        Normal


Добавим функции из предыдущего семинара, для оценки MSE и MSA

In [67]:
def calculate_mse(y_true, y_pred):
    n = len(y_true)
    mse = sum((y_true[i] - y_pred[i]) ** 2 for i in range(n)) / n

    return mse


def calculate_mae(y_true, y_pred):
    n = len(y_true)
    mae = sum(abs(y_true[i] - y_pred[i]) for i in range(n)) / n

    return mae

Реализуем метод наименьших квадратов

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   SalePrice      1460 non-null   int64 
 1   GrLivArea      1460 non-null   int64 
 2   GarageArea     1460 non-null   int64 
 3   OverallQual    1460 non-null   int64 
 4   Street         1460 non-null   object
 5   SaleCondition  1460 non-null   object
dtypes: int64(4), object(2)
memory usage: 68.6+ KB


так как помещений с нулём ванных комнат нет в датафрейме, то заменим пустые значения на нуль

Проверим наличие дубликатов

In [69]:
duplicate_rows_data = df[df.duplicated()]
print("number of duplicate rows: ", duplicate_rows_data.shape)
# Вывод дубликатов

duplicates = df[df.duplicated(keep=False)]  # keep=False показывает все дубликаты

print("\nДубликаты в DataFrame:")

print(duplicates)

number of duplicate rows:  (1, 6)

Дубликаты в DataFrame:
     SalePrice  GrLivArea  GarageArea  OverallQual Street SaleCondition
603     151000       1200         480            7   Pave        Normal
831     151000       1200         480            7   Pave        Normal


In [70]:
df = df.drop_duplicates()

In [71]:
duplicates = df[df.duplicated(keep=False)]  # keep=False показывает все дубликаты

print("\nДубликаты в DataFrame:")

print(duplicates)


Дубликаты в DataFrame:
Empty DataFrame
Columns: [SalePrice, GrLivArea, GarageArea, OverallQual, Street, SaleCondition]
Index: []


Дубликатов больше нет

Построим модель и обучим её


In [72]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
X = df[['GrLivArea', 'GarageArea', 'OverallQual']]
y = df["SalePrice"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создание и обучение модели
lr = LinearRegression()
lr.fit(X_train, y_train)

# Предсказания для обучающей выборки
y_train_pred = lr.predict(X_train)

# Предсказания для тестовой выборки
y_test_pred = lr.predict(X_test)

# Приведение y_train и y_train_pred к одному индексу
y_train = y_train.reset_index(drop=True)
y_train_pred = pd.Series(y_train_pred)

# Вычисление ошибок для обучающей выборки
mse_train = mean_squared_error(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)

# Приведение y_test и y_test_pred к одному индексу
y_test = y_test.reset_index(drop=True)
y_test_pred = pd.Series(y_test_pred)

mse_test = mean_squared_error(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

print(f'Обучающая выборка:\nMSE: {mse_train:.10f}\nMAE: {mae_train:.10f}\n')
print(f'Тестовая выборка:\nMSE: {mse_test:.10f}\nMAE: {mae_test:.10f}')

Обучающая выборка:
MSE: 1769627933.6199524403
MAE: 27296.8758083552

Тестовая выборка:
MSE: 1140151688.2083835602
MAE: 25525.7943690295


А что если мы преобразуем тип Object к числам и добавим эти атрибуты в модель?

In [73]:
unique_values = df['Street'].unique()

print(f"\nУникальные значения в столбце 'Street': {unique_values}")



Уникальные значения в столбце 'Street': ['Pave' 'Grvl']


In [74]:
unique_Condition = df['SaleCondition'].unique()

print(f"\nУникальные значения в столбце 'SaleCondition': {unique_Condition}")


Уникальные значения в столбце 'SaleCondition': ['Normal' 'Abnorml' 'Partial' 'AdjLand' 'Alloca' 'Family']


In [ ]:
data = {
    'Street': ['Pave', 'Grvl', 'Pave', 'Grvl'],
    'SaleCondition': ['Normal', 'Abnorml', 'Partial', 'Normal']
}
street_mapping = {'Pave': 1, 'Grvl': 0}
sale_condition_mapping = {
    'Normal': 1,
    'Abnorml': 2,
    'Partial': 3,
    'AdjLand': 4,
    'Alloca': 5,
    'Family': 6
}
# Замена значений в столбце 'Street'
df['Street'] = df['Street'].replace(street_mapping)
# Замена значений в столбце 'SaleCondition'
df['SaleCondition'] = df['SaleCondition'].replace(sale_condition_mapping)
# Вывод измененного DataFrame
print("\nИзмененный DataFrame:")
print(df)


Измененный DataFrame:
      SalePrice  GrLivArea  GarageArea  OverallQual  Street  SaleCondition
0        208500       1710         548            7       1              1
1        181500       1262         460            6       1              1
2        223500       1786         608            7       1              1
3        140000       1717         642            7       1              2
4        250000       2198         836            8       1              1
...         ...        ...         ...          ...     ...            ...
1455     175000       1647         460            6       1              1
1456     210000       2073         500            6       1              1
1457     266500       2340         252            7       1              1
1458     142125       1078         240            5       1              1
1459     147500       1256         276            5       1              1

[1459 rows x 6 columns]


C:\Users\Klimov_as\AppData\Local\Temp\ipykernel_20332\2973963258.py:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Street'] = df['Street'].replace(street_mapping)
C:\Users\Klimov_as\AppData\Local\Temp\ipykernel_20332\2973963258.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['SaleCondition'] = df['SaleCondition'].replace(sale_condition_mapping)


In [77]:
X = df.drop(["SalePrice"], axis=1)
y = df["SalePrice"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создание и обучение модели
lr = LinearRegression()
lr.fit(X_train, y_train)

# Предсказания для обучающей выборки
y_train_pred = lr.predict(X_train)

# Предсказания для тестовой выборки
y_test_pred = lr.predict(X_test)

# Приведение y_train и y_train_pred к одному индексу
y_train = y_train.reset_index(drop=True)
y_train_pred = pd.Series(y_train_pred)

# Вычисление ошибок для обучающей выборки
mse_train = mean_squared_error(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)

# Приведение y_test и y_test_pred к одному индексу
y_test = y_test.reset_index(drop=True)
y_test_pred = pd.Series(y_test_pred)

mse_test = mean_squared_error(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

print(f'Обучающая выборка:\nMSE: {mse_train}\nMAE: {mae_train}\n')
print(f'Тестовая выборка:\nMSE: {mse_test}\nMAE: {mae_test}')

Обучающая выборка:
MSE: 1763931294.8281713
MAE: 27230.76885721421

Тестовая выборка:
MSE: 1149676755.3149831
MAE: 25593.213533677535


Практически ничего не изменилось, видимо целевая переменная слабо кореллирует с добавленными атрибутами